In [ ]:
import numpy as np
import torch

from shapely.plotting import plot_polygon

import matplotlib.pyplot as plt
import matplotlib.style as style

style.use(
    "https://raw.githubusercontent.com/dominik-strutz/dotfiles/main/mystyle.mplstyle"
)
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# Cmap with only blue and deacreasig alpha
cmap1 = LinearSegmentedColormap.from_list(
    "mycmap",
    [
        "cornflowerblue",
        "mediumblue",
    ],
)

blue_cmap = cmap1
blue_cmap = blue_cmap(np.arange(blue_cmap.N))
blue_cmap[:, -1] = np.linspace(0, 1, blue_cmap.shape[0]) * 0.8
blue_cmap[:, :3] = [0.0, 0.0, 1.0]
blue_cmap = ListedColormap(blue_cmap)


cmap2 = LinearSegmentedColormap.from_list(
    "mycmap",
    [
        "salmon",
        "firebrick",
    ],
)

red_cmap = cmap2
red_cmap = red_cmap(np.arange(red_cmap.N))
red_cmap[:, -1] = np.linspace(0, 1, red_cmap.shape[0]) * 1.0
# red_cmap[:, :3] = [1.0, 0.0, 0.0]
red_cmap = ListedColormap(red_cmap)

In [ ]:
from dased.optimisation import DASOptimizationProblem, DASArchipelago

In [ ]:
from cdv_setup import (
    topo_data,
    prior_dist,
    design_space_full,
    tomo_roi,
)

In [ ]:
cable_properties = {
    "elevation": topo_data,
    "cable_length": 2500,
    "spacing": 10,
    "fixed_points": np.array([[1570, 1330]]),
    "N_knots": 10,
    "k": 1,
    "signal_decay": 0.4,
}

optimisation_properties = {
    "bounds": [
        [design_space_full.bounds[0], design_space_full.bounds[2]],
        [design_space_full.bounds[1], design_space_full.bounds[3]],
    ],
    "spatial_constraints": design_space_full.envelope.difference(design_space_full),
}

In [ ]:
from dased.helpers.srcloc import (
    MagnitudeRelation,
    ForwardHomogeneous,
    DataLikelihood,
)

from dased.criteria import EIGCriterion, RaySensitivity

In [ ]:
reference_distance_loc = 1000.0

f_max = 10.0  # Hz

# Magnitude relations for P and S waves
distance_relation_P = MagnitudeRelation(
    log_coeff=-2,
    reference_distance=reference_distance_loc,
)
distance_relation_S = MagnitudeRelation(
    log_coeff=-2, reference_relation=distance_relation_P
)

# Forward models for P and S waves
forward_function_P = ForwardHomogeneous(
    velocity=2500,
    wave_type="P",
    distance_relation=distance_relation_P,
    incidence_max=np.deg2rad(30),
)
forward_function_S = ForwardHomogeneous(
    velocity=1500,
    wave_type="S",
    distance_relation=distance_relation_S,
    incidence_max=np.deg2rad(30),
)

# Data likelihoods for P, S, and joint PS
data_likelihood_P = DataLikelihood(
    forward_function=dict(P=forward_function_P),
    std_corr=0.05,
    std_uncorr=0.01,
    # cor_length=0.0,
    f_max=f_max,
)
data_likelihood_S = DataLikelihood(
    forward_function=dict(S=forward_function_S),
    std_corr=0.05,
    std_uncorr=0.01,
    # cor_length=0.0,
    f_max=f_max,
)
data_likelihood_PS = DataLikelihood(
    forward_function=dict(P=forward_function_P, S=forward_function_S),
    std_corr=0.05,
    std_uncorr=0.01,
    # cor_length=0.0,
    f_max=f_max,
)

torch.manual_seed(0)  # For reproducibility
prior_samples = prior_dist.sample((1000,))

In [ ]:
##############################################################################
##############################################################################
############### THIS HAS TO BE FIRST PLOT IN THE NOTEBOOK
############### OTHERWISE IT WILL NOT WORK, FOR SOME REASON
##############################################################################
##############################################################################

from interactive_proposal import select_points

proposal_rayleigh = select_points(
    design_space=design_space_full,
    cable_properties=cable_properties,
    filename="data/optimisation/proposal_points_single_rayleigh.pkl",
)

proposal_eigp = select_points(
    design_space=design_space_full,
    cable_properties=cable_properties,
    filename="data/optimisation/proposal_points_single_eigp.pkl",
)

proposal_points = proposal_rayleigh + proposal_eigp

##############################################################################
############### IF THIS IS CALLED IT WILL NOT WORK AGAIN
############### BUT IT IS NEEDED TO HAVE LATE PLOTS DISPLAYED
##############################################################################

%matplotlib inline

In [ ]:
dx = dy = 20

x_range = (tomo_roi.buffer(50).bounds[0], tomo_roi.buffer(50).bounds[2])
y_range = (tomo_roi.buffer(50).bounds[1], tomo_roi.buffer(50).bounds[3])

n_x = int((x_range[1] - x_range[0]) / dx)
n_y = int((y_range[1] - y_range[0]) / dy)

print(f"n_x = {n_x}, n_y = {n_y}")


criteria_dict = dict(
    Rayleigh_tomo=RaySensitivity(
        data_type="rayleigh",
        n_points=(n_x, n_y),
        reference_distance=1_000.0,
        x_range=x_range,
        y_range=y_range,
        roi=tomo_roi,
        criterion="D",
        criterion_kwargs=dict(
            threshold=1e-4, penalty=10 * np.log(1e-4), normalize=True
        ),
    ),
    EIG_p=EIGCriterion(
        prior_samples[:1000],
        data_likelihood=data_likelihood_P,
        eig_method="NMC",
        N=1000,
        downsample=10,
    ),
)

In [ ]:
udp = DASOptimizationProblem(
    criteria_dict,
    verbose=0,
    **cable_properties,
    **optimisation_properties,
)

In [ ]:
archipelago = DASArchipelago(udp, n_islands=16, population_size=1028, verbose=1)

archipelago.initialize(
    proposal_points=proposal_points,
    perturb_proposal=50.0,
    perturb_knots=25.0,
    corr_len=500,
    corr_str=0.8,
    min_length=1000,
    show_progress=True,
    filename="data/optimisation/initial_population_multi",
)

In [ ]:
# initial_layouts = archipelago.get_current_layouts()

In [ ]:
# fig, ax = plt.subplots(figsize=(8, 8), dpi=120)

# plot_polygon(
#     design_space_full, ax=ax, facecolor='none', edgecolor='k', linewidth=2, label='Design Space',
#     add_points=False,
# )

# # Plot fixed points
# fixed_point = cable_properties['fixed_points'][0]
# ax.plot(fixed_point[0], fixed_point[1], 'go', markersize=8, label='Fixed Point')

# # Plot proposal points
# for i, pp in enumerate(proposal_points):
#     pp = np.array(pp)
#     ax.plot(pp[:, 0], pp[:, 1], 'o--', markersize=4,
#            label='Proposal Points' if i == 0 else '')

# # Plot initial layouts
# for layout in initial_layouts[:500]:  # Show fewer layouts for clarity
#     layout.plot(ax=ax, plot_style='line', color='k', alpha=0.3, linewidth=1.0, label='_nolegend_', zorder=-10)
# ax.plot([], [], color='k', alpha=0.3, linewidth=1.0, label='Initial Layouts')

# ax.set_xlabel('Easting (m)')
# ax.set_ylabel('Northing (m)')
# ax.set_title('DAS Layout Optimization - Initial Population')
# ax.grid(True, alpha=0.3)
# ax.set_aspect('equal')

# # Move legend below plot
# ax.legend(loc='upper center', bbox_to_anchoror=(0.5, -0.12), ncol=3, frameon=False)

# plt.tight_layout()
# plt.show()

In [ ]:
archipelago.get_file_info("data/optimisation/optimiser_multi")

In [ ]:
archipelago.optimize(
    n_generations=1000,
    migrate_every=20,
    show_progress=True,
    filename="data/optimisation/optimiser_multi",
)

In [ ]:
archipelago.plot_fitness_history()

In [ ]:
fitness_history = archipelago.get_fitness_history()
pareto_front = archipelago.get_pareto_front()[1]
best_fitness = archipelago.get_best_multi(method="compromise")[1]

fig, ax = plt.subplots(figsize=(6, 4), dpi=130)


# Use Wistia colormap to show progression over generations
cmap = plt.cm.bone_r
# generations = [0, 20, 40, 80, 160, 320]
generations = [0, 20, 40, 80, 160]

norm = plt.Normalize(-200, 700)

for idx, gen in enumerate(generations):
    fit_hist = fitness_history[gen]
    ax.scatter(
        fit_hist[:, 0], fit_hist[:, 1], color=[cmap(norm(gen))], edgecolor="k", s=15.0
    )
    # Plot horizontal and vertical lines to represent the Pareto front with a staircase pattern
    sorted_points = fit_hist[np.argsort(fit_hist[:, 0])]
    for i in range(len(sorted_points) - 1):
        # Horizontal line
        ax.plot(
            [sorted_points[i, 0], sorted_points[i + 1, 0]],
            [sorted_points[i + 1, 1], sorted_points[i + 1, 1]],
            color=cmap(norm(gen)),
            lw=1.0,
            zorder=-1,
        )
        # Vertical line
        ax.plot(
            [sorted_points[i, 0], sorted_points[i, 0]],
            [sorted_points[i, 1], sorted_points[i + 1, 1]],
            color=cmap(norm(gen)),
            lw=1.0,
            zorder=-1,
        )

ax.scatter(
    pareto_front[:, 0],
    pareto_front[:, 1],
    c=[cmap(norm(500))],
    label="Best Fitness",
    s=20,
    zorder=10,
)

ax.scatter(
    best_fitness[0],
    best_fitness[1],
    c=[cmap(norm(500))],
    label="Best Fitness",
    s=40,
    edgecolor="r",
    zorder=10,
)

# Plot horizontal and vertical lines to represent the Pareto front with a staircase pattern
sorted_points = pareto_front[np.argsort(pareto_front[:, 0])]
for i in range(len(sorted_points) - 1):
    # Horizontal line
    ax.plot(
        [sorted_points[i, 0], sorted_points[i + 1, 0]],
        [sorted_points[i + 1, 1], sorted_points[i + 1, 1]],
        color=cmap(norm(gen)),
        lw=1.0,
        zorder=-1,
    )
    # Vertical line
    ax.plot(
        [sorted_points[i, 0], sorted_points[i, 0]],
        [sorted_points[i, 1], sorted_points[i + 1, 1]],
        color=cmap(norm(gen)),
        lw=1.0,
        zorder=-1,
    )

cbar = plt.colorbar(
    plt.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=ax,
    ticks=generations,
    label="Generation",
    shrink=0.8,
)
cbar.ax.set_ylim(0, 200)

ax.set_xlabel("Rayleigh RER")
ax.set_ylabel("EIG (P)")
ax.set_title("Pareto Front (Best Fitness)")
# ax.set_aspect('equal')
fig.tight_layout()
plt.show()

In [ ]:
n = 5

top_layouts, top_fitness = archipelago.get_n_spread_multi(n)

mosaic = [["pareto"] * n, ["layout_{}".format(i) for i in range(n)]]
fig, axes = plt.subplot_mosaic(
    mosaic,
    figsize=(6, 6),
    dpi=130,
    gridspec_kw={"height_ratios": [1, 0.5]},
)

# --- Pareto front plot (leftmost) ---
ax = axes["pareto"]

# Use Wistia colormap to show progression over generations
cmap = plt.cm.bone_r
# generations = [0, 20, 40, 80, 160, 320]
generations = [0, 50, 200]

norm = plt.Normalize(-200, 700)

for idx, gen in enumerate(generations):
    fit_hist = fitness_history[gen]
    ax.scatter(
        fit_hist[:, 0], fit_hist[:, 1], color=[cmap(norm(gen))], edgecolor="k", s=15.0
    )
    # Plot horizontal and vertical lines to represent the Pareto front with a staircase pattern
    sorted_points = fit_hist[np.argsort(fit_hist[:, 0])]
    for i in range(len(sorted_points) - 1):
        # Horizontal line
        ax.plot(
            [sorted_points[i, 0], sorted_points[i + 1, 0]],
            [sorted_points[i + 1, 1], sorted_points[i + 1, 1]],
            color=cmap(norm(gen)),
            lw=1.0,
            zorder=-1,
        )
        # Vertical line
        ax.plot(
            [sorted_points[i, 0], sorted_points[i, 0]],
            [sorted_points[i, 1], sorted_points[i + 1, 1]],
            color=cmap(norm(gen)),
            lw=1.0,
            zorder=-1,
        )

    # add text with generation number below lowest point
    min_point = sorted_points[np.argmin(sorted_points[:, 1])]
    ax.text(
        min_point[0],
        min_point[1] - 0.02,
        f"Gen {gen}",
        fontsize=6,
        ha="center",
        va="top",
        color=cmap(norm(gen)),
    )

ax.scatter(pareto_front[:, 0], pareto_front[:, 1], c=[cmap(norm(500))], s=20, zorder=10)

ax.text(0.75, 5.2, "Pareto Front", fontsize=8, ha="center", va="bottom", color="k")


# ax.scatter(
#     best_fitness[0], best_fitness[1], c=[cmap(norm(500))],
#     s=40, edgecolor='r', zorder=10
# )


# Plot horizontal and vertical lines to represent the Pareto front with a staircase pattern
sorted_points = pareto_front[np.argsort(pareto_front[:, 0])]
for i in range(len(sorted_points) - 1):
    # Horizontal line
    ax.plot(
        [sorted_points[i, 0], sorted_points[i + 1, 0]],
        [sorted_points[i, 1], sorted_points[i, 1]],
        color=cmap(norm(500)),
        lw=1.0,
        zorder=-1,
    )
    # Vertical line
    ax.plot(
        [sorted_points[i + 1, 0], sorted_points[i + 1, 0]],
        [sorted_points[i, 1], sorted_points[i + 1, 1]],
        color=cmap(norm(500)),
        lw=1.0,
        zorder=-1,
    )

# Highlight the selected top designs
for i, (x, y) in enumerate(top_fitness):
    ax.scatter(x, y, marker="o", c="crimson", s=60, edgecolor="k", zorder=10)

ax.set_xlabel("Rayleigh D-optimality")
ax.set_ylabel("P-wave EIG")
# ax.legend(fontsize=9, loc='lower left')
# ax.grid(True, linestyle=':', alpha=0.5)

# --- Layouts plots (right) ---
for idx, (layout, fit) in enumerate(zip(top_layouts, top_fitness)):
    ax_layout = axes[f"layout_{idx}"]
    # Topography contours
    # contour = ax_layout.contour(
    #     topo_data.x, topo_data.y, topo_data.values.T,
    #     colors='dimgray', alpha=0.5,
    #     levels=[1600, 1800, 2000, 2200, 2400, 2600],
    #     linewidths=1.0
    # )
    # Design space
    plot_polygon(
        design_space_full,
        ax=ax_layout,
        facecolor="none",
        edgecolor="k",
        linestyle="--",
        linewidth=1.0,
        label="design space",
        add_points=False,
    )
    # Layout
    layout.plot(ax=ax_layout, plot_style="line", color="k", linewidth=1.5)

    ax_layout.set_xlim(400, 1700)
    ax_layout.set_ylim(600, 1700)

    ax_layout.set_xticks([])
    ax_layout.set_yticks([])
    ax_layout.set_aspect("equal")
    ax_layout.set_xlabel(
        f"Rayleigh: {fit[0]:.2f}\nP-wave: {fit[1]:.2f}",
        fontsize=8,
    )


fig.tight_layout()

fig.savefig("figures/pareto_front_layouts_multi.png", dpi=300, bbox_inches="tight")
fig.savefig("figures/pareto_front_layouts_multi.pdf", dpi=300, bbox_inches="tight")

plt.show()